In [100]:
import pandas as pd
import pandas as pd
import numpy as np 
import re
from matplotlib import pyplot as plt
from transformers import pipeline
from sklearn.model_selection import train_test_split
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from torch.optim import AdamW
from transformers import get_scheduler
from torch.utils.data import Dataset


#1 Food Product Group vs Primary Food Proudct Group. Any cell where they differ? 

#2 Do huggingface models update transformer weights, or just weights in the last MLP layer? Or something else?

#3 Train it!!!

# Basic Idea. 

If training is fine, then just update head. (fine place to start) But will probebly want to train all the parameters if possible and training is not doing well.

In [134]:
food_data = pd.read_excel('CGFP_bulk.xlsx')
product_name = food_data['Product Name']

In [135]:
food_data = food_data[['Product Type', 'Food Product Group']]
food_data.rename(columns={'Product Type': 'text', 'Food Product Group': 'labels'}, inplace=True)

In [136]:
food_data

,text,labels
0,beef patty 2 oz,Meat
1,APPLE-FUJI (80/90 CT) SORT 5 LB,Produce
2,BERRY-STRAWBERRY LAS CASE,Produce
3,BERRY-STRAWBERRY LAS CASE,Produce
4,Muffin Top Sweet Potato Choc 63218,Condiments & Snacks
...,...,...
154593,"FR, BLUEBERRY PARFAIT",Milk & Dairy
154594,"FR, STRAWBERRY PARFAIT",Milk & Dairy
154595,"FR, MIXED BERRY PARFAIT",Milk & Dairy
154596,"YOGURT, VNL GREEK SS CUP REF",Milk & Dairy


In [137]:
print("number of unique labels", len(food_data['labels'].unique()))
print("number of unique observations", len(food_data['text'].unique()))
print("number of total observations", len(food_data['text']))

number of unique labels 10
number of unique observations 82550
number of total observations 154598


In [138]:
food_data['labels'].unique()


array(['Meat', 'Produce', 'Condiments & Snacks', 'Milk & Dairy',
       'Beverages', 'Bread, Grains & Legumes', 'Meals', 'Seafood',
       'Non-Food', nan], dtype=object)

In [139]:
unique_products = food_data['text'].value_counts()
unique_products

APPLES FANCY RED/GREEN 125SZ                          107
TURKEY SALAMI SLICED 0.5                              100
TURKEY BOLOGNA SLICED 0.5                              99
TURKEY COOKED SLICED DARK CURED                        99
PEARS GREEN OR RED 100CT                               86
                                                     ... 
Uncured Bacon Ends & Pieces FRZ                         1
Uncured Maple Bacon Ends & Pieces FRZ                   1
Cured Bacon ends & pieces (frozen 30-lb. case) FRZ      1
4:1 Beef Frank FS Bulk Frozen                           1
DANISH MINI GKY CRANBERRY RTB BRD                       1
Name: text, Length: 82549, dtype: int64

Notes: Further analysis on split Product Names

Fine tune the lang model to categorize product types into product names after anaylsis on tags.

Detect whetehr info is metadata or useful information.

In [ ]:
labels = set()
for l in product_name:
    try:
        label = l.split(',')
        labels.update(label)
    except:
        labels.add(l)

In [117]:
train_df, test_df = train_test_split(food_data, test_size=0.15, random_state=42)

In [118]:
print(len(train_df), len(test_df))

131408 23190


In [129]:
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        tensor_sample = torch.tensor(sample, dtype=torch.float32)  # Convert to tensor
        return tensor_sample


In [130]:
dataset = CustomDataset(train_df.values)

In [131]:
train_dataloader = DataLoader(dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(test_df, batch_size=8)

In [133]:
for i in train_dataloader:
    pass

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [81]:
model_name = "distilbert-base-uncased" 
num_classes = 10
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_classes)
optimizer = AdamW(model.parameters(), lr=5e-5)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'classifier.w

In [83]:
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [84]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [91]:


progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        pass
        # batch = {k: v.to(device) for k, v in batch.items()}
        # outputs = model(**batch)
        # loss = outputs.loss
        # loss.backward()

        # optimizer.step()
        # lr_scheduler.step()
        # optimizer.zero_grad()
        # progress_bar.update(1)

  0%|          | 0/49278 [00:28<?, ?it/s]


KeyError: 42016

In [43]:
class TextClassifier:
    def __init__(self, model_name, num_classes):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_classes)

    def preprocess_text(self, text):
        inputs = self.tokenizer(text, padding=True, truncation=True, return_tensors="pt")
        return inputs

    def predict(self, text):
        inputs = self.preprocess_text(text)
        outputs = self.model(**inputs)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1).item()
        return predicted_class



In [64]:
model_name = "distilbert-base-uncased" 
num_classes = 10
classifier = TextClassifier(model_name, num_classes)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'classifier.w

In [141]:
text = train_df['text'].iloc[4]
inputs = classifier.preprocess_text(text)
print(inputs)

{'input_ids': tensor([[  101, 11225,  3059,  3585,  1062, 24475,  3263,  1013,  1012,  4008,
          2509, 11472,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [142]:
outputs = classifier.model(**inputs)

In [143]:
logits = outputs.logits
predicted_class = torch.argmax(logits, dim= 1).item()
print(predicted_class)

7


In [48]:
model_name = "distilbert-base-uncased" 
num_classes = 10
classifier = TextClassifier(model_name, num_classes)

#Text
text = train_df['text'].iloc[4]
print(text)

# Classify the text
predicted_class = classifier.predict(text)

# Map the predicted class index to the corresponding label
class_labels = ['Meat', 'Produce', 'Condiments & Snacks', 'Milk & Dairy',
       'Beverages', 'Bread, Grains & Legumes', 'Meals', 'Seafood',
       'Non-Food', 'Unknown']

predicted_label = class_labels[predicted_class]

print(f"Predicted class: {predicted_label}")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'classifier.w

Dressing Italian Golden ZTF 200/.443 Oz
Predicted class: Seafood


In [76]:
train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

NameError: name 'small_train_dataset' is not defined